# Research Article Briefing Agent

A **LangGraph**-based AI agent for discovering and summarizing academic papers.

**Designed for:** Information Systems researchers studying AI agent evaluation in healthcare / data science.

## Pipeline
```
START
  → [Gate 1] Set date range + research topic + batch size
  → Search academic papers (Tavily)
  → [Gate 2a] Review papers in batches, select papers to include
  → [Gate 2b] Set briefing focus prompt + word limit
  → Generate structured briefings (LLM)
  → [Gate 3] Review briefings, approve or request revisions
END
```

## Briefing Format (per paper)
```
**Title:** ...
**Domain:** ...

[Paragraph 1: Background, motivation, and problem addressed]

[Paragraph 2: Methods, key findings, and implications]
```

## Setup Checklist
1. Paste your API keys in **Cell 2**
2. Adjust models / settings in **Cell 3**
3. Run all cells in order
4. Interact at each Human Gate (type your input and press Enter)

---
## 1. Install Dependencies

In [ ]:
!pip install -q langchain-openai langgraph tavily-python python-dotenv pydantic typing-extensions

---
## 2. API Keys

- **LLM_API_KEY** — OpenRouter key ([openrouter.ai](https://openrouter.ai)) or your provider's key
- **LLM_BASE_URL** — base URL for your LLM provider (`https://openrouter.ai/api/v1` for OpenRouter)
- **TAVILY_API_KEY** — get one at [tavily.com](https://tavily.com) (free tier available)

In [ ]:
import os

os.environ["LLM_API_KEY"]  = ""   # ← paste your LLM API key here
os.environ["LLM_BASE_URL"] = "https://openrouter.ai/api/v1"  # ← change if not OpenRouter
os.environ["TAVILY_API_KEY"] = ""  # ← paste your Tavily API key here

assert os.environ["LLM_API_KEY"],    "⚠ Please fill in LLM_API_KEY above"
assert os.environ["TAVILY_API_KEY"], "⚠ Please fill in TAVILY_API_KEY above"
print("API keys set.")

---
## 3. Configuration

In [ ]:
CFG = {
    # ── LLM models ────────────────────────────────────────────────
    # Change model IDs to match your provider.
    # OpenRouter examples: "openai/gpt-4o", "anthropic/claude-3.5-sonnet", "google/gemini-flash-1.5"
    "models": {
        "search_agent": "openai/gpt-4o",   # used for query reformulation (optional, currently unused)
        "brief_agent":  "openai/gpt-4o",   # used for briefing generation
    },

    # ── Search defaults ───────────────────────────────────────────
    "default_date_from":    "2024-01",
    "default_date_to":      "2026-01",
    "default_topic":        "AI agent evaluation healthcare data science",
    "default_papers_per_batch": 5,       # how many papers to show at once (1–10)
    "max_papers_to_fetch":  20,           # total papers fetched from Tavily

    # ── Briefing defaults ─────────────────────────────────────────
    "default_briefing_prompt": "summarize key contributions, methodology, and implications",
    "default_word_limit":      300,       # target words per briefing

    # ── Control ───────────────────────────────────────────────────
    "max_revisions":  3,                  # max briefing revision rounds
}

print("Configuration loaded.")
for k, v in CFG.items():
    print(f"  {k}: {v}")

---
## 4. Imports, LLM Setup & Search Client

In [ ]:
import os
import json
import operator
import textwrap
from pathlib import Path
from datetime import datetime
from typing import Annotated, Literal
from typing_extensions import TypedDict

from langchain_openai import ChatOpenAI
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langgraph.graph import StateGraph, START, END
from langgraph.types import interrupt, Command
from langgraph.checkpoint.memory import MemorySaver
from IPython.display import display, Markdown, HTML


# ── LLM factory ──────────────────────────────────────────────────────────────
def _make_llm(model_id: str, max_tokens: int = 4096) -> ChatOpenAI:
    return ChatOpenAI(
        model=model_id,
        base_url=os.environ.get("LLM_BASE_URL", "https://openrouter.ai/api/v1"),
        api_key=os.environ.get("LLM_API_KEY", ""),
        max_tokens=max_tokens,
        max_retries=3,
    )

llm_brief = _make_llm(CFG["models"]["brief_agent"], max_tokens=8192)

# ── Tavily search client ──────────────────────────────────────────────────────
from tavily import TavilyClient
_tavily = TavilyClient(api_key=os.environ.get("TAVILY_API_KEY", ""))

print("\u2713 LLM and search client initialized")
print(f"  Brief agent:  {CFG['models']['brief_agent']}")

---
## 5. Academic Paper Search Tool

In [ ]:
# Academic domains to search — Tavily will prioritize these sources
ACADEMIC_DOMAINS = [
    "arxiv.org",
    "semanticscholar.org",
    "pubmed.ncbi.nlm.nih.gov",
    "dl.acm.org",
    "ieeexplore.ieee.org",
    "springer.com",
    "nature.com",
    "sciencedirect.com",
    "researchgate.net",
    "scholar.google.com",
    "wiley.com",
    "tandfonline.com",
    "jamanetwork.com",
    "bmj.com",
    "plos.org",
]


def search_academic_papers(
    topic: str,
    date_from: str,
    date_to: str,
    max_results: int = 20,
) -> list[dict]:
    """
    Search for academic papers using Tavily with academic domain filtering.

    Args:
        topic:       Research topic keywords
        date_from:   Start date string, e.g. '2024-11'
        date_to:     End date string,   e.g. '2026-01'
        max_results: Maximum number of papers to return

    Returns:
        List of dicts with keys: index, title, url, abstract, source
    """
    # Build query — include date context and academic keywords
    query = f"{topic} research paper academic {date_from} {date_to}"
    print(f"  [Search] Query: {query[:100]}")

    try:
        results = _tavily.search(
            query=query,
            max_results=max_results,
            search_depth="advanced",
            include_domains=ACADEMIC_DOMAINS,
        )
        papers = []
        for i, r in enumerate(results.get("results", [])):
            url = r.get("url", "")
            papers.append({
                "index":    i + 1,
                "title":    r.get("title", "Unknown Title"),
                "url":      url,
                "abstract": r.get("content", "")[:600].strip(),
                "source":   url.split("/")[2] if url else "unknown",
                "score":    round(r.get("score", 0.0), 3),
            })
        return papers

    except Exception as e:
        print(f"  [WARNING] Search failed: {e}")
        return []


print("\u2713 Search tool defined")

---
## 6. State Schema

In [ ]:
class BriefingAgentState(TypedDict):
    """Full state of the Research Briefing Agent."""

    # ── Phase 1: Search parameters ────────────────────────────────
    date_from:          str   # e.g. "2024-11"
    date_to:            str   # e.g. "2026-01"
    search_topic:       str   # e.g. "patient EHR AI agent evaluation"
    papers_per_batch:   int   # 1–10, how many papers to display at once

    # ── Phase 2: Search results ───────────────────────────────────
    all_papers:         list  # All found papers: [{index, title, url, abstract, source}]
    shown_up_to:        int   # Pagination cursor (how many papers have been shown)
    selected_indices:   list  # 1-based paper indices chosen by human

    # ── Phase 3: Briefing parameters ─────────────────────────────
    briefing_prompt:    str   # What aspect to focus on
    word_limit:         int   # Target word count per briefing

    # ── Outputs ───────────────────────────────────────────────────
    briefings:          list  # Generated briefing texts (one per selected paper)

    # ── Human feedback at each gate ───────────────────────────────
    hf_search:          str   # Gate 1 feedback
    hf_papers:          str   # Gate 2a feedback ("more" | "search_again" | "selected")
    hf_briefing:        str   # Gate 3 feedback

    # ── Control ───────────────────────────────────────────────────
    revision_round:     int
    status:             str
    messages:           Annotated[list, operator.add]


print("\u2713 State schema defined")

---
## 7. Agent Nodes

In [ ]:
# ============================================================
# GATE 1 — Search Setup
# Human sets: date range | topic | papers per batch
# ============================================================

def search_setup_gate(state: dict) -> dict:
    """Interrupt: collect search parameters from the human."""
    print("\n" + "=" * 70)
    print("GATE 1: Search Setup")
    print("=" * 70)

    prompt = textwrap.dedent(f"""\
        ## Research Paper Search Setup

        Please enter your search parameters in **pipe-separated** format:

        ```
        <date_from> | <date_to> | <topic> | <papers_per_batch>
        ```

        | Field              | Example                                      |
        |--------------------|----------------------------------------------|
        | `date_from`        | `2024-11`  (November 2024)                  |
        | `date_to`          | `2026-01`  (January 2026)                   |
        | `topic`            | `patient EHR AI agent evaluation`           |
        | `papers_per_batch` | `5`  (1–10 papers shown at a time)          |

        **Example input:**
        ```
        2024-11 | 2026-01 | AI agent evaluation in healthcare EHR | 5
        ```

        *Press Enter to use defaults: `{CFG['default_date_from']} | {CFG['default_date_to']} | {CFG['default_topic']} | {CFG['default_papers_per_batch']}`*
    """)

    feedback = interrupt(prompt)
    feedback = str(feedback).strip()

    # Use defaults if blank
    if not feedback or feedback.lower() in ("approve", "approved", "ok", "default"):
        feedback = (
            f"{CFG['default_date_from']} | {CFG['default_date_to']} "
            f"| {CFG['default_topic']} | {CFG['default_papers_per_batch']}"
        )

    # Parse pipe-separated input
    parts = [p.strip() for p in feedback.split("|")]
    date_from = parts[0] if len(parts) > 0 and parts[0] else CFG["default_date_from"]
    date_to   = parts[1] if len(parts) > 1 and parts[1] else CFG["default_date_to"]
    topic     = parts[2] if len(parts) > 2 and parts[2] else CFG["default_topic"]
    try:
        batch = max(1, min(10, int(parts[3]))) if len(parts) > 3 else CFG["default_papers_per_batch"]
    except (ValueError, IndexError):
        batch = CFG["default_papers_per_batch"]

    print(f"  Date range:  {date_from} → {date_to}")
    print(f"  Topic:       {topic}")
    print(f"  Batch size:  {batch}")

    return {
        "date_from":          date_from,
        "date_to":            date_to,
        "search_topic":       topic,
        "papers_per_batch":   batch,
        "hf_search":          "approved",
        "shown_up_to":        0,
        "selected_indices":   [],
        "all_papers":         [],
        "briefings":          [],
        "revision_round":     0,
        "status":             "searching",
        "messages":           [AIMessage(content=f"Searching for: *{topic}* ({date_from} → {date_to})")],
    }

In [ ]:
# ============================================================
# NODE — Search Papers
# ============================================================

def do_search(state: dict) -> dict:
    """Call Tavily to fetch academic papers matching the topic and date range."""
    print("\n" + "=" * 70)
    print("NODE: Searching for Academic Papers")
    print("=" * 70)

    topic     = state.get("search_topic",  CFG["default_topic"])
    date_from = state.get("date_from",     CFG["default_date_from"])
    date_to   = state.get("date_to",       CFG["default_date_to"])

    papers = search_academic_papers(
        topic=topic,
        date_from=date_from,
        date_to=date_to,
        max_results=CFG["max_papers_to_fetch"],
    )

    print(f"  Found {len(papers)} papers")
    for p in papers[:3]:
        print(f"    [{p['index']}] {p['title'][:70]}...")
    if len(papers) > 3:
        print(f"    ... and {len(papers) - 3} more")

    return {
        "all_papers":  papers,
        "shown_up_to": 0,
        "messages":    [AIMessage(content=f"Found **{len(papers)}** papers on *{topic}*.")],
    }

In [ ]:
# ============================================================
# GATE 2a — Paper Review & Selection
# Shows papers in batches; human selects papers to include
# ============================================================

def paper_review_gate(state: dict) -> dict:
    """Interrupt: display current batch of papers; human selects or asks for more."""
    print("\n" + "=" * 70)
    print("GATE 2a: Paper Review & Selection")
    print("=" * 70)

    all_papers  = state.get("all_papers", [])
    shown_up_to = state.get("shown_up_to", 0)
    batch_size  = state.get("papers_per_batch", CFG["default_papers_per_batch"])
    total       = len(all_papers)

    # Slice the current batch
    batch     = all_papers[shown_up_to : shown_up_to + batch_size]
    batch_end = min(shown_up_to + batch_size, total)

    # ── Build display content ────────────────────────────────────
    if not all_papers:
        content = "\u26a0\ufe0f No papers were found. Please try different search parameters.\n\nType `search: <new topic>` to search again."
    elif not batch:
        content = f"\u26a0\ufe0f All **{total}** papers have been shown.\n\n"
        content += "Please select papers to include (e.g., `1,3,5`) or type `search: <new topic>` to search again."
    else:
        content = f"## Papers Found — Showing {shown_up_to+1}\u2013{batch_end} of {total}\n\n"
        for p in batch:
            content += f"---\n"
            content += f"**{p['index']}. {p['title']}**\n\n"
            content += f"- **Source:** {p['source']}  \n"
            content += f"- **URL:** {p['url']}  \n"
            content += f"- **Abstract:** {p['abstract'][:350]}...\n\n"

        content += "---\n"
        content += "**Commands:**\n"
        content += "- Select papers: type numbers separated by commas, e.g. `1,3,5`\n"
        if batch_end < total:
            content += "- See next batch: type `more`\n"
        content += "- Search again: type `search: <new topic>`\n"
        content += f"\n*({total - batch_end} more papers available)*" if batch_end < total else ""

    feedback = interrupt(content)
    feedback = str(feedback).strip()
    print(f"  Input: {feedback[:80]}")

    # ── Parse feedback ──────────────────────────────────────────
    if feedback.lower() == "more":
        # Show next batch
        return {
            "shown_up_to": shown_up_to + batch_size,
            "hf_papers":   "more",
        }

    elif feedback.lower().startswith("search:"):
        # New search
        new_topic = feedback[7:].strip()
        print(f"  New topic: {new_topic}")
        return {
            "search_topic": new_topic,
            "hf_papers":    "search_again",
            "shown_up_to":  0,
            "all_papers":   [],
        }

    else:
        # Parse paper selection — e.g. "1,3,5" or "1 3 5" or "1-3"
        # Support comma and space separation
        tokens = feedback.replace(",", " ").split()
        indices = []
        for tok in tokens:
            # Handle ranges like "1-3"
            if "-" in tok:
                try:
                    a, b = tok.split("-", 1)
                    indices.extend(range(int(a), int(b) + 1))
                except ValueError:
                    pass
            elif tok.isdigit():
                indices.append(int(tok))

        # Validate indices
        valid_indices = [p["index"] for p in all_papers]
        indices = [i for i in indices if i in valid_indices]

        # Default: select all shown papers if no valid input
        if not indices:
            indices = [p["index"] for p in batch]
            print(f"  No valid selection parsed — defaulting to shown batch: {indices}")
        else:
            print(f"  Selected indices: {indices}")

        return {
            "selected_indices": indices,
            "hf_papers":        "selected",
        }

In [ ]:
# ============================================================
# GATE 2b — Briefing Parameter Setup
# Human sets: focus prompt | word limit
# ============================================================

def briefing_setup_gate(state: dict) -> dict:
    """Interrupt: human sets briefing focus and word count."""
    print("\n" + "=" * 70)
    print("GATE 2b: Briefing Parameters")
    print("=" * 70)

    all_papers       = state.get("all_papers", [])
    selected_indices = state.get("selected_indices", [])
    selected_papers  = [p for p in all_papers if p["index"] in selected_indices]

    # Build list of selected paper titles
    titles_md = "\n".join(
        f"  {i+1}. {p['title']}"
        for i, p in enumerate(selected_papers)
    )

    content = textwrap.dedent(f"""\
        ## Briefing Parameters

        **You selected {len(selected_papers)} paper(s):**
        {titles_md}

        ---

        Please set briefing parameters in **pipe-separated** format:

        ```
        <focus prompt> | <word limit>
        ```

        | Field          | Description                                               | Example |
        |----------------|-----------------------------------------------------------|---------|
        | `focus prompt` | What to explain in the briefing                           | `explain the methodology, key findings, and clinical impact` |
        | `word limit`   | Target words per briefing (recommended: 250–500)          | `350` |

        **Example input:**
        ```
        focus on evaluation methodology and clinical implications | 350
        ```

        *Press Enter for defaults: `{CFG['default_briefing_prompt']} | {CFG['default_word_limit']}`*
    """)

    feedback = interrupt(content)
    feedback = str(feedback).strip()

    # Use defaults if blank
    if not feedback or feedback.lower() in ("approve", "approved", "ok", "default"):
        feedback = f"{CFG['default_briefing_prompt']} | {CFG['default_word_limit']}"

    parts = [p.strip() for p in feedback.split("|")]
    briefing_prompt = parts[0] if parts and parts[0] else CFG["default_briefing_prompt"]
    try:
        word_limit = int(parts[1]) if len(parts) > 1 else CFG["default_word_limit"]
        word_limit = max(100, min(1000, word_limit))  # clamp 100–1000
    except (ValueError, IndexError):
        word_limit = CFG["default_word_limit"]

    print(f"  Focus prompt: {briefing_prompt}")
    print(f"  Word limit:   {word_limit} words per briefing")

    return {
        "briefing_prompt": briefing_prompt,
        "word_limit":       word_limit,
    }

In [ ]:
# ============================================================
# NODE — Generate Briefings
# LLM generates structured briefings for each selected paper
# ============================================================

BRIEF_SYSTEM_PROMPT = textwrap.dedent("""\
    You are an expert academic research summarizer specializing in
    Information Systems, AI/ML, and healthcare informatics.

    For EACH paper you receive, write a structured briefing using
    EXACTLY this format (no deviations):

    ---
    **Title:** [full paper title]
    **Domain:** [1–3 word research domain, e.g., "Healthcare AI", "Clinical NLP",
                 "EHR Systems", "AI Agent Evaluation", "Data Science"]

    [Paragraph 1 — 2–4 sentences: background, motivation, and the specific
     research problem or gap this paper addresses.]

    [Paragraph 2 — 2–4 sentences: methods, key results, and contributions or
     implications for the field.]

    ---

    Rules:
    - Stay within the requested word limit.
    - Be specific and evidence-based; avoid vague language.
    - Incorporate any revision feedback if provided.
    - Focus on: {focus}
""")


def generate_briefings(state: dict) -> dict:
    """Generate structured briefings for all selected papers using the LLM."""
    print("\n" + "=" * 70)
    print("NODE: Generating Briefings")
    print("=" * 70)

    all_papers       = state.get("all_papers", [])
    selected_indices = state.get("selected_indices", [])
    selected_papers  = [p for p in all_papers if p["index"] in selected_indices]

    briefing_prompt = state.get("briefing_prompt", CFG["default_briefing_prompt"])
    word_limit      = state.get("word_limit",      CFG["default_word_limit"])
    revision_round  = state.get("revision_round",  0)
    prior_feedback  = state.get("hf_briefing",     "")

    print(f"  Papers:    {len(selected_papers)}")
    print(f"  Focus:     {briefing_prompt}")
    print(f"  Words:     ~{word_limit} per briefing")
    if revision_round > 0:
        print(f"  Revision:  round {revision_round} (feedback: {prior_feedback[:60]})")

    system_prompt = BRIEF_SYSTEM_PROMPT.format(focus=briefing_prompt)

    all_briefings = []
    for i, paper in enumerate(selected_papers):
        title_short = paper["title"][:65]
        print(f"  [{i+1}/{len(selected_papers)}] {title_short}...")

        feedback_note = (
            f"\n\nRevision feedback from human reviewer: {prior_feedback}"
            if prior_feedback and prior_feedback != "approved"
            else ""
        )

        user_msg = textwrap.dedent(f"""\
            Paper {i+1}:
            Title:    {paper['title']}
            Source:   {paper['source']}
            URL:      {paper['url']}
            Abstract: {paper['abstract']}
            {feedback_note}

            Write a briefing of approximately {word_limit} words.
        """)

        try:
            resp = llm_brief.invoke([
                SystemMessage(content=system_prompt),
                HumanMessage(content=user_msg),
            ])
            briefing_text = resp.content.strip()
        except Exception as e:
            print(f"    [WARNING] LLM call failed: {e}")
            briefing_text = (
                f"---\n**Title:** {paper['title']}\n"
                f"**Domain:** Unknown\n\n[Generation failed: {e}]\n---"
            )

        all_briefings.append(briefing_text)
        word_count = len(briefing_text.split())
        print(f"    \u2713 {word_count} words generated")

    print(f"\n  Total: {len(all_briefings)} briefings")

    return {
        "briefings": all_briefings,
        "status":    "pending_review",
        "messages":  [AIMessage(content=f"Generated **{len(all_briefings)}** briefings.")],
    }

In [ ]:
# ============================================================
# GATE 3 — Final Review
# Human reviews briefings, approves or requests revisions
# ============================================================

def final_review_gate(state: dict) -> dict:
    """Interrupt: display all generated briefings for human approval."""
    print("\n" + "=" * 70)
    print("GATE 3: Final Review")
    print("=" * 70)

    briefings      = state.get("briefings", [])
    revision_round = state.get("revision_round", 0)
    max_rev        = CFG["max_revisions"]

    # Compose full briefing display
    if briefings:
        all_briefings_text = "\n\n".join(briefings)
    else:
        all_briefings_text = "_No briefings were generated._"

    revision_note = (
        f"\n\n> \u26a0\ufe0f Max revisions ({max_rev}) reached — this is the final version."
        if revision_round >= max_rev
        else f"\n\n> Revision round {revision_round}/{max_rev} — you may request {max_rev - revision_round} more revision(s)."
    )

    content = textwrap.dedent(f"""\
        ## Generated Briefings ({len(briefings)} paper(s))

        {all_briefings_text}

        ---
        {revision_note}

        **Options:**
        - Press Enter or type `approve` → save and finish
        - Type feedback → revise all briefings (e.g., "make paragraph 2 more concise")
        - Type `reselect` → go back to paper selection
    """)

    feedback = interrupt(content)
    feedback = str(feedback).strip()

    # Approved?
    is_approved = not feedback or feedback.lower() in (
        "approve", "approved", "ok", "yes", "lgtm", "looks good"
    )

    if is_approved or revision_round >= max_rev:
        if not is_approved:
            print(f"  ** Max revisions ({max_rev}) reached — finalizing **")
        print("  \u2192 Approved")
        return {"hf_briefing": "approved", "status": "complete"}

    if feedback.lower() == "reselect":
        print("  \u2192 Going back to paper selection")
        return {
            "hf_briefing":      "reselect",
            "selected_indices": [],
            "briefings":        [],
            "shown_up_to":      0,
        }

    print(f"  \u2192 Revision requested (round {revision_round+1}): {feedback[:80]}")
    return {
        "hf_briefing":   feedback,
        "revision_round": revision_round + 1,
    }

---
## 8. Routing Functions

In [ ]:
def route_after_paper_review(
    state: dict,
) -> Literal["paper_review_gate", "do_search", "briefing_setup_gate"]:
    """After Gate 2a: show more papers, search again, or proceed to briefing setup."""
    hf = state.get("hf_papers", "")
    if hf == "more":
        return "paper_review_gate"     # show next batch
    elif hf == "search_again":
        return "do_search"              # new search with updated topic
    else:
        return "briefing_setup_gate"   # papers selected → set briefing params


def route_after_final_review(
    state: dict,
) -> Literal["generate_briefings", "paper_review_gate", "__end__"]:
    """After Gate 3: approve → end, reselect → paper review, else → revise."""
    hf = state.get("hf_briefing", "")
    if hf == "approved":
        return "__end__"
    elif hf == "reselect":
        return "paper_review_gate"
    else:
        return "generate_briefings"


print("\u2713 Routing functions defined")

---
## 9. Build Graph

In [ ]:
def build_graph(checkpointer=None):
    """
    Build the Research Briefing Agent graph.

    Graph structure:
        START
          → search_setup_gate   [Gate 1: date + topic + batch]
          → do_search           [Tavily search]
          → paper_review_gate   [Gate 2a: show papers, get selection]
              ↑ (more / search_again loop)
          → briefing_setup_gate [Gate 2b: focus prompt + word limit]
          → generate_briefings  [LLM generates briefings]
          → final_review_gate   [Gate 3: approve or revise]
              ↓ (revise loop / reselect loop)
        END
    """
    g = StateGraph(BriefingAgentState)

    # ── Register nodes ──────────────────────────────────────────
    g.add_node("search_setup_gate",   search_setup_gate)
    g.add_node("do_search",           do_search)
    g.add_node("paper_review_gate",   paper_review_gate)
    g.add_node("briefing_setup_gate", briefing_setup_gate)
    g.add_node("generate_briefings",  generate_briefings)
    g.add_node("final_review_gate",   final_review_gate)

    # ── Phase 1: Search setup ────────────────────────────────────
    g.add_edge(START, "search_setup_gate")
    g.add_edge("search_setup_gate", "do_search")

    # ── Phase 2a: Paper review loop ──────────────────────────────
    g.add_edge("do_search", "paper_review_gate")
    g.add_conditional_edges(
        "paper_review_gate",
        route_after_paper_review,
        {
            "paper_review_gate":   "paper_review_gate",   # show more
            "do_search":           "do_search",            # search again
            "briefing_setup_gate": "briefing_setup_gate", # proceed
        },
    )

    # ── Phase 2b → 3: Briefing generation ───────────────────────
    g.add_edge("briefing_setup_gate", "generate_briefings")
    g.add_edge("generate_briefings",  "final_review_gate")
    g.add_conditional_edges(
        "final_review_gate",
        route_after_final_review,
        {
            "generate_briefings": "generate_briefings",  # revise
            "paper_review_gate":  "paper_review_gate",   # reselect
            "__end__":            END,                   # done
        },
    )

    return g.compile(checkpointer=checkpointer)


# Build the graph
agent = build_graph(checkpointer=MemorySaver())

print("=" * 60)
print("GRAPH BUILT — Research Briefing Agent")
print("=" * 60)
print(f"  Nodes: {len(agent.get_graph().nodes)}")
print(f"  Edges: {len(agent.get_graph().edges)}")

---
## 10. Run the Agent (Interactive)

The cell below runs the full pipeline.

- At each **Human Gate**, the agent will display content as rendered Markdown and wait for your input.
- **Press Enter** (or type `approve`) to accept.
- **Type feedback or commands** to navigate (see each gate's instructions).

In [ ]:
import time

# ── Fresh run configuration ──────────────────────────────────────────────────
config     = {"configurable": {"thread_id": f"briefing-{datetime.now().strftime('%Y%m%d_%H%M%S')}"}}
gate_num   = 0

# ── Start the agent ──────────────────────────────────────────────────────────
print("Starting Research Briefing Agent...")
print("=" * 70)
t0 = time.time()

result = agent.invoke({}, config)

# ── Interactive loop: handle each interrupt gate ─────────────────────────────
while True:
    snapshot = agent.get_state(config)
    if not snapshot.next:
        break  # graph completed

    # Render the interrupt content
    for task in snapshot.tasks:
        if hasattr(task, "interrupts"):
            for intr in task.interrupts:
                gate_num += 1
                display(HTML("<hr style='border:3px solid #2E75B6; margin:24px 0 8px 0'>"))
                display(Markdown(f"### \u25b6 Human Gate {gate_num}"))
                display(Markdown(str(intr.value)))
                display(HTML("<hr style='border:1px solid #ccc; margin:8px 0 16px 0'>"))

    # Get human input
    feedback = input("> ").strip()
    if not feedback:
        feedback = "approved"

    # Resume the graph
    result = agent.invoke(Command(resume=feedback), config)

# ── Completion ───────────────────────────────────────────────────────────────
elapsed = time.time() - t0
display(HTML("<hr style='border:4px solid #1B2A4A; margin:24px 0'>"))
display(Markdown("## \u2705 Agent Complete"))
print(f"Elapsed: {elapsed:.0f}s")

---
## 11. View & Save Results

In [ ]:
# ── Display final briefings ───────────────────────────────────────────────────
briefings = result.get("briefings", [])

if briefings:
    display(Markdown(f"## Research Briefings ({len(briefings)} papers)"))
    for i, brief in enumerate(briefings):
        display(HTML(f"<h4>Briefing {i+1}</h4>"))
        display(Markdown(brief))
        display(HTML("<hr>"))
else:
    print("No briefings were generated.")

In [ ]:
import yaml

# ── Output directory ──────────────────────────────────────────────────────────
ts = datetime.now().strftime("%Y%m%d_%H%M%S")
output_dir = Path(ts)
output_dir.mkdir(parents=True, exist_ok=True)

briefings       = result.get("briefings", [])
search_topic    = result.get("search_topic", "unknown")
selected_papers = [
    p for p in result.get("all_papers", [])
    if p["index"] in result.get("selected_indices", [])
]

# ── Markdown report ───────────────────────────────────────────────────────────
md_path = output_dir / f"briefings_{ts}.md"
with open(md_path, "w", encoding="utf-8") as f:
    f.write(f"# Research Briefings\n\n")
    f.write(f"**Topic:** {search_topic}  \n")
    f.write(f"**Date range:** {result.get('date_from', '?')} – {result.get('date_to', '?')}  \n")
    f.write(f"**Generated:** {datetime.now().isoformat()}  \n")
    f.write(f"**Papers:** {len(briefings)}  \n")
    f.write(f"**Focus prompt:** {result.get('briefing_prompt', '?')}  \n\n")
    f.write("---\n\n")
    for i, brief in enumerate(briefings):
        f.write(brief)
        f.write("\n\n---\n\n")
print(f"Markdown saved: {md_path}")

# ── Metadata YAML ─────────────────────────────────────────────────────────────
meta_path = output_dir / "meta.yaml"
with open(meta_path, "w", encoding="utf-8") as f:
    yaml.dump({
        "timestamp":      datetime.now().isoformat(),
        "topic":          search_topic,
        "date_from":      result.get("date_from", ""),
        "date_to":        result.get("date_to", ""),
        "briefing_prompt":result.get("briefing_prompt", ""),
        "word_limit":     result.get("word_limit", 0),
        "total_papers_found":   len(result.get("all_papers", [])),
        "selected_papers":      len(briefings),
        "model":          CFG["models"]["brief_agent"],
        "elapsed_seconds": round(elapsed, 1),
        "papers": [
            {"index": p["index"], "title": p["title"], "url": p["url"]}
            for p in selected_papers
        ],
    }, f, default_flow_style=False, sort_keys=False, allow_unicode=True)
print(f"Metadata saved: {meta_path}")

# ── List saved files ──────────────────────────────────────────────────────────
print(f"\nOutput directory: {output_dir.resolve()}")
for p in sorted(output_dir.iterdir()):
    print(f"  {p.name:45s} {p.stat().st_size:>8,} bytes")